<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0


    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:


      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-23 16:20:59
-------------------
qualified stocks: 94
with latest results: 32
still star stocks: 18


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.99 L
Current:  1.28 C
-------------------
Today PnL: 78.05 K (0.61%)
Current PnL: -16.88 L (-11.97%)
CY Booked + Current PnL: -12.59 L (-8.93%)
-------------------
Total profit:  3.60 L
Total loss:  -20.49 L
-------------------
Total Booked + Current PnL: 14.39 L (12.62%)
Total Booked PnL: 31.27 L (27.42%)
Curr Year Booked PnL: 4.29 L (3.35%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 78.60 L (61.37%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.40 C (109.12%)
Deployed:  1.14 C
Current:  1.28 C
CAGR/XIRR %: 8.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBILIFE,1496.49,1924.99,1.70,H-LC,86.17,100727.0,16924.0,7071.0,2.24,20.19,7.02,28.63,159.0,2.39,0.82,34.60,AR,ATH,INSURANCE
62,RELIANCE,1291.83,1526.00,4.61,X-LC,81.91,155521.0,14712.0,10809.0,1.23,10.45,6.95,18.13,157.0,1.36,1.26,22.80,XY25,NTT,REFINERIES
4,ADANIPORTS,1103.69,1583.00,-1.13,M-LC,91.49,99073.0,20711.0,13315.0,1.75,26.43,13.44,43.43,39.0,1.56,0.80,32.51,XY24,NTT,MISC
56,PIDILITIND,2504.06,3576.00,-17.89,X-LC,26.60,90951.0,15829.0,16326.0,1.24,21.07,17.95,42.81,100.0,0.97,0.74,14.06,X40,BTT,CHEMICALS
69,SBIN,760.30,863.00,-14.39,M-LC,29.26,209482.0,8002.0,19209.0,0.67,3.97,9.17,13.51,71.0,0.42,1.70,17.07,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VBL,500.38,672.85,-13.36,H-LC,1.06,149069.0,-3547.0,56154.0,4.09,-2.32,37.67,34.47,12.0,-0.06,1.21,12.30,AR,ATH,FMCG
37,ICICIPRULI,600.83,791.05,-10.22,H-MC,65.96,145137.0,9349.0,33643.0,3.62,6.89,23.18,31.66,184.0,0.28,1.18,19.68,X40,ATH,INSURANCE
75,STARHEALTH,551.56,761.00,2.71,M-SC,93.62,168087.0,-32129.0,108164.0,3.39,-16.05,64.35,37.97,110.0,-0.30,1.37,34.96,XY24,NTT,INSURANCE
45,JPPOWER,18.73,26.20,-43.26,L-SC,14.89,114958.0,-26884.0,83460.0,3.27,-18.95,72.60,39.88,136.0,-0.32,0.93,16.50,XY24,NTT,POWER
5,ALKYLAMINE,2347.98,4567.52,-15.28,X-SC,25.53,78909.0,-22054.0,117496.0,3.21,-21.84,148.90,94.53,75.0,-0.19,0.64,16.99,SR,ATH,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-55.73,L-SC,95.74,64397.0,-49152.0,89203.0,-3.65,-43.29,138.52,35.27,182.0,-0.55,0.52,57.61,XR,NTT,HOTELS
87,VAIBHAVGBL,350.21,521.0,57.58,X-SC,61.70,104954.0,-47037.0,121159.0,-2.72,-30.95,115.44,48.77,45.0,-0.39,0.85,20.55,XR,NTT,JEWELLERY
73,SIS,477.00,528.0,2085.38,M-SC,50.00,59160.0,-21930.0,30598.0,-2.10,-27.04,51.72,10.69,162.0,-0.72,0.48,19.90,X40N,NTT,MISC
13,BERGEPAINT,558.54,680.0,-18.10,H-MC,8.51,175903.0,-1154.0,39649.0,-2.03,-0.65,22.54,21.75,191.0,-0.03,1.43,25.83,XY24,NTT,PAINTS
88,VALIANTORG,512.64,838.0,-330.82,H-SC,68.09,98033.0,-35253.0,119845.0,-1.87,-26.45,122.25,63.47,135.0,-0.29,0.80,64.33,XR,NTT,CHEMICALS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,AWL,342.88,485.00,-69.50,H-MC,4.26,165068.0,-52661.0,142899.0,0.95,-24.19,86.57,41.45,1.0,-0.37,1.34,10.27,XY24,NTT,FMCG
38,INDIAMART,2327.09,4911.36,-51.84,H-SC,51.06,125509.0,2173.0,134797.0,-1.03,1.76,107.40,111.05,2.0,0.02,1.02,22.98,AR,ATH,MISC
81,TCS,3888.13,4998.00,-15.84,X-LC,48.94,210876.0,-22412.0,89011.0,1.08,-9.61,42.21,28.55,3.0,-0.25,1.71,8.76,X40,BTT,IT
41,INFY,1432.81,2275.00,-12.78,X-LC,67.02,217507.0,18346.0,98726.0,1.12,9.21,45.39,58.78,4.0,0.19,1.77,13.56,X40,BTT,IT
80,TATAMOTORS,771.32,1065.00,-52.64,M-LC,75.53,210447.0,-15550.0,101604.0,0.11,-6.88,48.28,38.07,5.0,-0.15,1.71,23.89,XY24,NTT,AUTO


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,2.57,H-SC,98.94,118399.0,20752.0,26924.0,0.16,21.25,22.74,48.82,185.0,0.77,0.96,31.53,X40,ATH,PHARMA
68,SBILIFE,1496.49,1924.99,1.70,H-LC,86.17,100727.0,16924.0,7071.0,2.24,20.19,7.02,28.63,159.0,2.39,0.82,34.60,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-17.89,X-LC,26.60,90951.0,15829.0,16326.0,1.24,21.07,17.95,42.81,100.0,0.97,0.74,14.06,X40,BTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,-1.13,M-LC,91.49,99073.0,20711.0,13315.0,1.75,26.43,13.44,43.43,39.0,1.56,0.80,32.51,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,45.26,M-SC,84.04,150594.0,13016.0,81396.0,1.14,9.46,54.05,68.62,60.0,0.16,1.22,30.56,X40N,NTT,TEXTILES
57,POLYCAB,5005.95,7541.35,5.29,H-MC,58.51,100020.0,14919.0,28186.0,-1.08,17.53,28.18,50.65,44.0,0.53,0.81,25.73,X40N,ATH,CABLES
55,PFIZER,4245.50,6318.30,2.57,H-SC,98.94,118399.0,20752.0,26924.0,0.16,21.25,22.74,48.82,185.0,0.77,0.96,31.53,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,0.94,H-SC,62.77,148826.0,10966.0,26789.0,0.33,7.95,18.00,27.39,87.0,0.41,1.21,28.99,X40,ATH,FMCG
4,ADANIPORTS,1103.69,1583.00,-1.13,M-LC,91.49,99073.0,20711.0,13315.0,1.75,26.43,13.44,43.43,39.0,1.56,0.80,32.51,XY24,NTT,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-3.33,H-SC,10.64,108766.0,6762.0,38264.0,0.05,6.63,35.18,44.14,31.0,0.18,0.88,23.95,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-31.87,M-MC,31.91,133214.0,4062.0,83006.0,0.52,3.15,62.31,67.42,51.0,0.05,1.08,29.56,XR,NTT,BANKS
38,INDIAMART,2327.09,4911.36,-51.84,H-SC,51.06,125509.0,2173.0,134797.0,-1.03,1.76,107.40,111.05,2.0,0.02,1.02,22.98,AR,ATH,MISC
93,WIPRO,243.46,420.00,-10.62,M-LC,44.68,153363.0,2418.0,107032.0,0.58,1.60,69.79,72.51,28.0,0.02,1.25,15.47,XR,NTT,IT
33,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,78.72,150808.0,-105.0,60564.0,0.76,-0.07,40.16,40.06,22.0,-0.00,1.23,23.35,AR,ATH,AUTO


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.00,-55.73,L-SC,95.74,64397.0,-49152.0,89203.0,-3.65,-43.29,138.52,35.27,182.0,-0.55,0.52,57.61,XR,NTT,HOTELS
52,MASFIN,326.60,399.50,-23.67,H-SC,57.45,86700.0,-11280.0,33154.0,0.09,-11.51,38.24,22.32,133.0,-0.34,0.70,25.75,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.00,-1.13,M-LC,91.49,99073.0,20711.0,13315.0,1.75,26.43,13.44,43.43,39.0,1.56,0.80,32.51,XY24,NTT,MISC
88,VALIANTORG,512.64,838.00,-330.82,H-SC,68.09,98033.0,-35253.0,119845.0,-1.87,-26.45,122.25,63.47,135.0,-0.29,0.80,64.33,XR,NTT,CHEMICALS
68,SBILIFE,1496.49,1924.99,1.70,H-LC,86.17,100727.0,16924.0,7071.0,2.24,20.19,7.02,28.63,159.0,2.39,0.82,34.60,AR,ATH,INSURANCE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VIPIND,488.80,718.00,-839.17,H-SC,100.00,70395.0,-24921.0,69614.0,-1.30,-26.15,98.89,46.89,156.0,-0.36,0.57,38.85,X40N,NTT,MISC
55,PFIZER,4245.50,6318.30,2.57,H-SC,98.94,118399.0,20752.0,26924.0,0.16,21.25,22.74,48.82,185.0,0.77,0.96,31.53,X40,ATH,PHARMA
78,TANLA,917.30,1963.11,-39.41,H-SC,97.87,148822.0,-90593.0,363542.0,1.43,-37.84,244.28,114.01,17.0,-0.25,1.21,36.49,AR,ATH,IT
92,WHIRLPOOL,1167.49,2270.00,-36.30,M-SC,96.81,185632.0,14011.0,148060.0,-0.34,8.16,79.76,94.43,24.0,0.09,1.51,37.01,X40,NTT,DURABLES
20,COFFEEDAY,59.14,80.00,-55.73,L-SC,95.74,64397.0,-49152.0,89203.0,-3.65,-43.29,138.52,35.27,182.0,-0.55,0.52,57.61,XR,NTT,HOTELS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.57
1,25,39.19
2,50,68.97


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    33.30
X40N    14.10
XY25    13.99
X40     13.84
AR      12.21
XR      10.14
X200     1.23
SR       1.16
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.89
M-SC    18.67
X-LC    16.12
H-MC     8.63
M-LC     7.98
H-LC     6.74
X-MC     5.84
M-MC     5.75
X-SC     2.88
L-SC     2.40
L-LC     1.21
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,12.44,-1.53,34.27
BANKS,9.52,-12.94,65.73
IT,9.52,-19.94,86.43
MISC,6.72,-30.45,88.15
INSURANCE,6.24,2.20,26.96
FINANCE,6.22,-24.71,83.92
PAINTS,5.65,-18.29,53.29
AUTO,4.64,-9.60,57.48
ELECTRICAL,4.46,-5.52,59.20


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2873932.0,26
AR,1213057.0,13
X40N,1210248.0,17
XR,1100847.0,14
X40,724237.0,11
XY25,471389.0,10
SR,202151.0,2
X200,64546.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2321605.0,21
M-SC,2015725.0,21
X-LC,709009.0,12
M-MC,558729.0,5
H-MC,463531.0,8
X-SC,353005.0,4
M-LC,323029.0,6
X-MC,317755.0,5
L-SC,301327.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      920007.0      6
M-SC       XY24      747033.0      6
H-SC       AR        558038.0      3
           X40N      507501.0      6
M-SC       X40N      458660.0      6
M-MC       XY24      393703.0      3
X-LC       X40       393097.0      5
M-SC       XR        296368.0      4
H-SC       XR        282346.0      4
M-SC       X40       216143.0      2
H-LC       AR        214052.0      4
M-LC       XY24      196788.0      4
H-MC       XY24      182548.0      2
L-SC       XR        153560.0      2
X-LC       AR        140614.0      2
X-MC       XY24      139139.0      2
X-SC       XR        121159.0      1
M-SC       XY25      119404.0      1
X-SC       SR        117496.0      1
           X40N      114350.0      2
X-LC       XY25      110752.0      4
M-LC       XR        107032.0      1
X-MC       X40N      101551.0      2
M-SC       AR         93462.0      1
           SR         84655.0      1
L-SC       XY24       83460.0      1
M-MC       XR         83006.0      1
           AR         82020.0      1
X-MC       XY25       77065.0      1
H-MC       XY25       73573.0      1
X-LC       X200       64546.0      1
L-SC       AR         64307.0      1
H-MC       X40        61284.0      2
           AR         60564.0      1
           XR         57376.0      1
H-SC       X40        53713.0      2
L-LC       XY25       40942.0      1
H-LC       XY25       30444.0      1
H-MC       X40N       28186.0      1
M-LC       XY25       19209.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
